In [2]:
import tensorflow as tf
import numpy
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Define our input values

In [103]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

## Deffining our initial weights

In [4]:
def weight_variable(shape):
    w_h = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(w_h)
def bias_variable(shape):
    b_h = tf.constant(0.1, shape = shape)
    return tf.Variable(b_h)

## Deffining useful functions

In [5]:
def conv2d(Input, Weights):
    return tf.nn.conv2d(Input, Weights, strides = [1, 1, 1, 1],
                        padding='SAME')
def max_pool_2x2(Input):
    return tf.nn.max_pool(Input, ksize = [1, 2, 2, 1],
                         strides = [1, 2, 2, 1], padding = 'SAME')

In [6]:
def cnn_layer(Input, Weights_shape, Bias_shape):
    w_conv = weight_variable(Weights_shape)
    b_conv = weight_variable(Bias_shape)
    h_conv = tf.nn.relu(conv2d(Input, w_conv) + b_conv)
    return h_conv, w_conv, b_conv

## CapsT

In [7]:
#Inputs = conv output
#n_convcaps = 8
#n_channels = 32
#kernel_size = 9
#n_strides = 2
#padding = 'valid' 


In [8]:
def conv2d(Input, Weights, n_strides=2, padding='SAME'):
    return tf.nn.conv2d(Input, Weights, strides = [1, n_strides, n_strides, 1], padding=padding)
def max_pool_2x2(Input):
    return tf.nn.max_pool(Input, ksize = [1, 2, 2, 1],
                         strides = [1, 2, 2, 1], padding = 'SAME')

In [121]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [10]:
def cnn_layer(Input, Weights_shape, Bias_shape, n_strides, padding, activation=True):
    w_conv = weight_variable(Weights_shape)
    b_conv = weight_variable(Bias_shape)
    if activation == True:
        h_conv = tf.nn.relu(conv2d(Input, w_conv, n_strides, padding) + b_conv)
    else:
        h_conv = conv2d(Input, w_conv, n_strides, padding) + b_conv
    return h_conv, w_conv, b_conv

In [97]:
def Primary_caps(nn_batch,Inputs, n_convcaps, n_channels, kernel_size, n_strides, padding):
    h_conv, W_conv, b_conv = cnn_layer(Inputs, kernel_size, [n_convcaps*n_channels], n_strides, padding, activation='False')
    q=tf.shape(Inputs)
    caps_outs = tf.reshape(h_conv, (nn_batch,-1 , n_convcaps))#(Inputs.shape[0], -1 , n_convcaps))
    #caps_outs = tf.reshape(h_conv, [(Inputs.shape)[0], -1, n_convcaps])
    return h_conv, caps_outs 

In [12]:
def squashing_func(vector, axis=-1):
    sq_norm = tf.reduce_sum(tf.square(vector),0, keep_dims=True);
    first_eq = sq_norm/(1+sq_norm);
    second_eq = vector / tf.sqrt(sq_norm);
    return first_eq*second_eq

In [13]:
def digcaps(Input, o_size, v_len):
    print Input.shape
    r_Input = tf.reshape(Input, [(Input.shape)[0], -1, 1, (Input.shape)[2]])
    b_I = tf.constant(np.zeros([1, Input.shape[1].value, o_size, 1], dtype=np.float32))
    print r_Input.shape, b_I.shape
    W = weight_variable([1, 1152, 10, 8, 16])
    print W.shape
    t_Input = tf.tile(Input,[1, 1, 10])
    W= tf.tile(W, [Input.shape[0], 1, 1, 1, 1])
    print t_Input.shape
    print W.shape
    return r_Input

In [94]:
def margin_loss(y_true, y_pred):
    """
    Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.
    :param y_true: [None, n_classes]
    :param y_pred: [None, num_capsule]
    :return: a scalar loss value.
    """
    L = y_true * tf.square(tf.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * tf.square(tf.maximum(0., y_pred - 0.1))

    return tf.reduce_mean(tf.reduce_sum(L, axis=1))

In [154]:
nn_batch = 50;
nn_routing = 3;
cfg = tf.app.flags.FLAGS
tf.Session.reset
with tf.device('/gpu:1'): #or '/cpu:0' for cpu only
    x_image = tf.reshape(x, [-1, 28, 28, 1])    
    h_conv1, W_conv1, b_conv1 = cnn_layer(x_image, [9, 9, 1, 256], [256], 1, padding='VALID')
    #h_conv, W_conv, b_conv = cnn_layer(h_conv1, [9, 9, 256, 32*8], [32*8], 2, padding='VALID', activation='False')
    h_pout, prim_caps = Primary_caps(nn_batch, h_conv1, 8, 32, [9, 9, 256, 32*8], 2,padding='VALID')
    primarycps_s= squashing_func(prim_caps);
    #digitCaps = digcaps(primarycaps_squashed, 10, 16);
    #h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    #b_ij = tf.constant(0, shape = [1, 10, 10, 1,1])
    #c = tf.nn.softmax(b_ij, dim =2)
    #print primarycps_s.shape[-2].value
    primarycps_s2 = tf.reshape(primarycps_s, shape=(nn_batch, -1, 1, primarycps_s.shape[-1].value))
    #print primarycps_s2
    bs = tf.constant(np.zeros([1, 1152, 10, 1, 1], dtype=np.float32))
    W = weight_variable([1, 1152, 10, 8, 16])
    W = tf.tile(W, [nn_batch, 1, 1, 1, 1])
    
    inputs_expanded = tf.expand_dims(tf.expand_dims(primarycps_s, 3), 2)
    primarycps_s3 = tf.tile(inputs_expanded, [1, 1, 10, 1, 1])
    
    u_hat = tf.matmul(W, primarycps_s3, transpose_a=True)
    
    for r in range(nn_routing):
        sft_max = tf.nn.softmax(bs, dim=2)
        sft_max = tf.tile(sft_max, [nn_batch, 1, 1,1, 1])
        mlt_sft_hat = tf.multiply(sft_max, u_hat)
        mlt_sft_hat = tf.reduce_sum(mlt_sft_hat, axis=1, keep_dims=True)
        squash_mlt = squashing_func(mlt_sft_hat)
        
        squash_mlt_r = tf.tile(squash_mlt, [1, 1152, 1, 1, 1])
        u_hat_v = tf.matmul(u_hat, squash_mlt_r, transpose_a=True)
        
        bs += tf.reduce_sum(u_hat_v, axis=0, keep_dims=True)
       
    
    squash_mlt = tf.squeeze(squash_mlt, axis=1)
    #Masking
    
    length_v = tf.sqrt(tf.reduce_sum(tf.square(squash_mlt), axis = 2,
                                    keep_dims = True)+1e-9)
    
    softmax_v = tf.nn.softmax(length_v, dim=1)
    argmax_idx = tf.to_int32(tf.argmax(softmax_v, axis = 1))
    argmax_idx = tf.reshape(argmax_idx, shape=(nn_batch,))
    y_=tf.to_float(y_)
    if not True:
        masked_vector = []
        for btch in range(nn_batch):
            v = squash_mlt[btch][argmax_idx[btch], :]
            masked_vector.append(tf.reshape(v, shape=(1,1,16,1)))
        masked_vector = tf.concat(masked_vector, axis=0)
       # print masked_vector.shape
    else:
        y_=tf.to_float(y_)
       # print y_.dtype
        masked_vector = tf.matmul(tf.squeeze(squash_mlt),
                                 tf.reshape(y_, (-1, 10, 1)), transpose_a=True)
        length_v = tf.sqrt(tf.reduce_sum(tf.square(squash_mlt), axis=2,
                                        keep_dims = True)+1e-9)
    #Decoder
    vector_j = tf.reshape(masked_vector, shape=(nn_batch, -1))
    W_fc1 = weight_variable([16, 512])
    b_fc1 = bias_variable([512])
    h_fc1 = tf.nn.relu(tf.matmul(vector_j, W_fc1) + b_fc1)
    
    W_fc2 = weight_variable([512, 1024])
    b_fc2 = bias_variable([1024])
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

    W_fc3 = weight_variable([1024, 784])
    b_fc3 = bias_variable([784])
    h_fc3 = tf.nn.sigmoid(tf.matmul(h_fc2, W_fc3) + b_fc3)
    
    recons_img = tf.reshape(h_fc3, shape=(nn_batch,28,28,1))
    #print h_fc3.shape, recons_img.shape
    #print length_v[0][:], y[0]
    
    #l_function = margin_loss(y_, h_fc3)
    max_L1 = tf.square(tf.maximum(0., 0.9 - length_v))
    max_L2 = tf.square(tf.maximum(0., length_v-0.1))
    max_L1 = tf.reshape(max_L1, shape=(nn_batch, -1))
    max_L2 = tf.reshape(max_L2, shape=(nn_batch, -1))   
    lambda_t = 0.5*(1-y_)
    Lc = y_*max_L1+lambda_t*max_L2
    margin_loss = tf.reduce_mean((tf.reduce_sum(Lc, axis=1)))
    
    rec_loss = tf.reshape(x, shape=(nn_batch, -1))
    rec_squared = tf.square(h_fc3-rec_loss)
    rec_error = tf.reduce_mean(rec_squared)
    
    total_loss = margin_loss + 0.0005*rec_error
    #print max_L1.shape, max_L2.shape, lambda_t.shape, Lc.shape
    #print margin_loss, total_loss
    train_step = tf.train.AdamOptimizer(1e-4).minimize(total_loss)
    
    
    correct_prediction = tf.equal(tf.argmax(tf.reshape(length_v, shape=(nn_batch, -1)), 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with tf.Session(config = tf.ConfigProto(allow_soft_placement = True, log_device_placement = True)) as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(20000):
            batch = mnist.train.next_batch(nn_batch)
            if i%100 ==0:
                train_accuracy = accuracy.eval(feed_dict={x:batch[0],
                                                         y_:batch[1]})
                print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})


                
                

ResourceExhaustedError: OOM when allocating tensor of shape [9,9,256,256] and type float
	 [[Node: Variable_899/Adam/Initializer/zeros = Const[_class=["loc:@Variable_899"], dtype=DT_FLOAT, value=Tensor<type: float shape: [9,9,256,256] values: [[[0 0 0]]]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op u'Variable_899/Adam/Initializer/zeros', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist

In [70]:
if not True:
    print 1
else:
    print 2

2


In [ ]:
        x = mnist.train.images[0:nn_batch]
        y_ = mnist.train.labels[0:nn_batch]
        xx = x_image
        hh = h_conv1
        h_pouts = h_pout
        prim = prim_caps
        squashed = primarycps_s
        bb = bs;
        ww = W;
        expanded = inputs_expanded
        p_s3 = primarycps_s3
        mult_u_hat = u_hat
        sft_max_s = sft_max 
        after_routing = squash_mlt
        what_is = length_v

print "Input image shape", x.shape
print "Input label shape", y_.shape
print "Output of first convolutional layer shape", hh.shape
print "Output of the first layer", h_pouts.shape
print "Reshaped output of the first layer", prim.shape
print "squashed output of the first layer", primarycps_s.shape
print "bias for the routing", bb.shape
print "weight for routing", ww.shape
print "expanded inputs", expanded.shape
print "expand the inputs for the 10 outputs", p_s3.shape
print "u_hat", mult_u_hat.shape
print "softmax shape", sft_max_s.shape
print "after routing shape", after_routing.shape
print what_is.shape